In [ ]:
import sys

sys.path.append("../")
from helper.utils import *

from helper.catboost import *
from config import *

random_seed_cpu(RANDOM_STATE)

In [ ]:
is_debug = False
nrows = 1000

In [ ]:
version = "v1"

In [ ]:
if is_debug:
    train = pd.read_csv(
        join(proc_data_path, "train_{}.csv".format(version)), nrows=nrows
    )
    test = pd.read_csv(
        join(proc_data_path, "test_{}.csv".format(version)), nrows=nrows / 2
    )
else:
    train = pd.read_csv(join(proc_data_path, "train_{}.csv".format(version)))
    test = pd.read_csv(join(proc_data_path, "test_{}.csv".format(version)))

In [ ]:
target_name = "target"
Id_name = "image_id"
prediction_names=["electron", "muon", "pion", "kaon", "proton"]
features_to_remove = [target_name, Id_name, "validation", "fold", "event_id",
'count_=_zero_row_0','arg_max_column_0',
 'arg_max_column_1',
 'arg_max_column_8',
 'arg_max_row_0',
 'arg_max_row_1','feat_2count', 'feat_3count', 'feat_4count', 'feat_36count', 'feat_95count'
                     ]+prediction_names
features = [
    feature for feature in train.columns.tolist() if feature not in features_to_remove
]
# data_characterization(train[features])

In [ ]:
from sklearn.metrics import log_loss


def metric(x, y):
    return log_loss(x, y)

[catboost params documentation](https://catboost.ai/docs/concepts/python-reference_parameters-list.html)

In [ ]:
params = {
    "loss_function": "MultiClass",
    "eval_metric": "MultiClass",
    "learning_rate": 0.1,
    "random_seed": RANDOM_STATE,
    "l2_leaf_reg": 3,
    "bagging_temperature": 1,  # 0 inf
    "rsm":0.9,
    "depth":6,
    "od_type": "Iter",
    "od_wait": 50,
    "thread_count": 8,
    "iterations": 50000,
    "verbose_eval": False,
    "use_best_model": True,
}

In [ ]:
cat_features = []
cat_features = [
    train[features].columns.get_loc(c) for c in cat_features if c in train[features]
]

In [ ]:
other_params = {
    "prediction_type": "Probability",  # it could be RawFormulaVal ,Class,Probability
    "cat_features": cat_features,
    "print_result": False,  # print result for a single model should be False whene use_kfold==True
    "plot_importance": False,  # plot importance for single model should be false whene use_kfold==True
    "predict_train": False,  # predict train for the single model funcation False only whene  use_kfold==True
    "num_class": 5,
    "target_name": target_name,
    "features": features,
    "metric": metric,
    "params": params,
    "use_kfold": True,  # condtion to use kfold or single model
    "plot_importance_kfold": True,  # plot importance after K fold train
    "print_kfold_eval": True,  # print evalation in kfold mode
    "weight":None,
    "print_time":True
}

In [ ]:
if other_params["use_kfold"]:
    oof_train, test_pred, final_train_score, oof_score, models = cat_train(
        train, test, other_params
    )
    validation=fill_predictions_df(train,oof_train,prediction_names)
    
else:
    train_pred, val_pred, test_pred, train_score, val_score, model = cat_train(
        train, test, other_params
    )
    validation=fill_predictions_df(train[train.validation==1],val_pred.reshape((-1,1)),prediction_names)
    

In [ ]:
score = metric(validation[target_name], validation[prediction_names].values)
print(score)
print("Train")
print("train mean ", train[target_name].mean(), "train std ", train[target_name].std())
print("oof mean ", np.mean(validation[prediction_names].values), "oof std ", np.std(validation[prediction_names].values))

In [ ]:
target_names = ["electron", "muon", "pion", "kaon", "proton"]
save_oof_multi_class(
    train,
    test,
    oof_train,
    test_pred,
    prediction_names,
    Id_name,
    oof_train_path,
    oof_test_path,
    score,
    model_name="cat",
)

In [ ]:
sub_name = "cat_{}_{}".format(round(score, 3), str("Kfold"))

In [ ]:
target_names = ["electron", "muon", "pion", "kaon", "proton"]
make_sub_multi_class(test, test_pred, Id_name, target_names, join(sub_path, sub_name))